In [29]:
# import own packages
from dog_features_extraction import extractFeat
from load_accel_data import loadAccel
from comparing_classifiers_ovo import classifier_comparison
# import system packages
import pandas as pd



dataDir = "data/all_dogs.mat"
label, accelData = loadAccel(dataDir)

#set number of data points grouped to extract features, n - i.e windowing
n = 5

#calculate spare data points so no null values are created in the features dataframe
remove = len(accelData)%n

for i in range(0, len(accelData)-remove, n):
    if (i==0): features = extractFeat(accelData[i:i+n], n)
    else: features = pd.concat([features,extractFeat(accelData[i:i+5], n)], axis = 0)
features = features.loc[0]
print(features.shape)




(42020, 45)


In [30]:

#select most common label for every n datapoints: y_features
for i in range(0, len(label)-remove, n):
    if(i==0):
        y_mode = label[i:i+n].mode()
        if(len(y_mode>1)): y_features = pd.DataFrame(data = y_mode.mean().round(0))
        else: y_features = pd.DataFrame(data = label[i:i+n].mode())
    else:
        y_mode = label[i:i+n].mode()
        if(len(y_mode>1)):
            y = pd.DataFrame(data = y_mode.mean().round(0))
            y_features = pd.concat([y_features, y], axis = 1)

        else:
            y_features = pd.concat([y_features, y_mode], axis = 1)
y_features = y_features.transpose()
y_features = y_features.astype('int32')
print(y_features.shape)




(42020, 1)


In [31]:
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

clf_rbf = svm.SVC(kernel ='rbf', decision_function_shape='ovo')
C_range = np.logspace(-2, 10, 13)
gamma_range = np.logspace(-9, 3, 13)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=2, test_size=0.25, random_state=0)
grid = GridSearchCV(clf_rbf, param_grid=param_grid, cv=cv)
grid.fit(features, y_features)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


KeyboardInterrupt: 

In [ ]:
classifier_comparison(features, y_features)
